In [1]:
import os
import pandas as pd

In [2]:
bam_files = [xx for xx in os.listdir() if xx.endswith('.bam')]
len(bam_files)

50

In [3]:
mut_dict = {
    '1168':'caf1d_ccr4d',
    '1022':'mot2d',
    '1023':'mot2d',
    '301':'swi6d',
    '302':'clr3d',
    '324':'chp2d',
    '491':'mit1d',
    '504':'rrp6d',
    '510':'caf1d',
    #'523':'unknown_1',
    '523':'unknown',
    #'524':'unknown_2',
    '524':'unknown',
    '530':'exo2d',
    '544':'ccr4d',
    '591':'caf1d',
    '638':'ago1d',
    '80':'clr4d',
    '63':'wt', 
    '65':'wt',
    'WT':'wt',
    'fake-reads':'fake-reads'
}

In [4]:
seq_dict = {
    'S2-ChIP':'S2ChIP', 
    'S2-ChIP-INPUT':'INPUT',
    'S2-ChIP-OIN':'INPUT',
    'H3K9me2':'H3K9me2'
}

In [5]:
bam_df = pd.DataFrame({'bam_file':bam_files})
bam_df["sample_id"] = bam_df["bam_file"].map(lambda x: x.split(".")[0])
bam_df[['mutant_id', 'seq_type', 'replicate']] = bam_df['sample_id'].str.split('_', n=3, expand=True)
bam_df['mutant_name'] = bam_df['mutant_id'].map(mut_dict) # map to mutant names
bam_df['seq_category'] = bam_df['seq_type'].map(seq_dict) # map to mutant names

In [6]:
bam_df = bam_df.sort_values(['seq_category', 'mutant_id', 'replicate']).reset_index(drop=True)
bam_df = bam_df.sort_values(['mutant_id', 'replicate']).reset_index(drop=True)
#bam_df

In [7]:
import string

In [8]:
template_string = (
    "track ${sample_id}\n"
    "bigDataUrl ${sample_id}.Aligned.sortedByCoord.out.bam\n"
    "shortLabel ${mutant_name} ${seq_category} ${replicate} reads\n"
    "longLabel ${mutant_name} ${seq_category} ${replicate} reads mapped with STAR\n"
    "type bam\n"
    "doWiggle on\n"
    "priority ${sample_idx}\n"
    "group ${seq_category}\n\n"
)

In [9]:
t = string.Template(template_string)

In [10]:
file1 = open('trackDb_py.txt', 'w')

In [11]:
#for rr in bam_df[0:2].itertuples():
for rr in bam_df.itertuples():
    result = t.substitute(
        sample_idx = str(rr.Index + 2),
        sample_id = rr.sample_id,
        mutant_name = rr.mutant_name,
        seq_category = rr.seq_category,
        replicate = rr.replicate,
    )
    #print(result)
    file1.write(result)

In [12]:
# Closing file
file1.close()

In [13]:
# # Checking if the data is
# # written to file or not
# file1 = open('myfile.txt', 'r')
# print(file1.read())
# file1.close()